# Predição de Satisfação de Clientes (Olist) - Treinamento
Este notebook contém o pipeline de pré-processamento, balanceamento e treinamento de 5 modelos de classificação para prever se um cliente ficará satisfeito (Nota 4-5) ou insatisfeito (Nota 1-3).

**Objetivo:** Construir e otimizar modelos de Machine Learning para classificar a satisfação dos clientes (Satisfeito vs. Insatisfeito).

**Dataset:** Dados processados do e-commerce brasileiro (Olist).

**Features selecionadas:** Preço, Frete, Dimensões do produto e status de Atraso.

## 1. Tratamento e Preparação de Dados

### 1.1. Organização e Carga

In [3]:
import pandas as pd

path = '../../../data/raw/'

orders = pd.read_csv(f'{path}olist_orders_dataset.csv')
items = pd.read_csv(f'{path}olist_order_items_dataset.csv')
reviews = pd.read_csv(f'{path}olist_order_reviews_dataset.csv')
products = pd.read_csv(f'{path}olist_products_dataset.csv')

### 1.2. Merge

In [5]:
# Para saber as datas de cada avaliação
df_class = pd.merge(reviews, orders, on='order_id', how='inner')

# Para saber preço e frete
df_class = pd.merge(df_class, items, on='order_id', how='inner')

# Para saber categoria e peso
df_class = pd.merge(df_class, products, on='product_id', how='inner')

print(f"Total de linhas para processar: {df_class.shape[0]}")

Total de linhas para processar: 112372


### 1.3. Target

Se tentarmos prever notas de 1 a 5, o modelo vai ter dificuldade porque a maioria das pessoas dá nota 5. Vamos simplificar para um problema binário: Satisfeito (1) vs Insatisfeito (0).

In [ ]:
date_cols = ['order_purchase_timestamp', 'order_delivered_customer_date', 'order_estimated_delivery_date']
for col in date_cols:
    df_class[col] = pd.to_datetime(df_class[col])

# Feature
df_class['atrasado'] = (df_class['order_delivered_customer_date'] > df_class['order_estimated_delivery_date']).astype(int)

# Nota 4 e 5 = 1 (Satisfeito) | Nota 1, 2 e 3 = 0 (Insatisfeito)
df_class['target'] = df_class['review_score'].apply(lambda x: 1 if x >= 4 else 0)

### 1.4. Limpeza e Features

Alguns pedidos podem não ter sido entregues ainda (data de entrega vazia), e alguns produtos podem estar sem peso ou tamanho cadastrado.

In [ ]:
# removendo linhas onde a entrega não aconteceu
df_class = df_class.dropna(subset=['order_delivered_customer_date'])

features_list = [
    'price', 
    'freight_value', 
    'product_weight_g', 
    'product_length_cm', 
    'product_height_cm', 
    'product_width_cm', 
    'atrasado'
]

X = df_class[features_list].copy()
y = df_class['target']

X = X.fillna(X.median())

print(f"Formato do X: {X.shape}")

Formato do X: (110012, 7)


### 1.5. Divisão de Treino/Teste e Escalonamento

Padronização dos dados utilizando StandardScaler para normalização das escalas (evitando, por exemplo, que o peso em gramas domine o preço em reais).

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### 1.6. Balanceamento

Como o dataset é desbalanceado (muito mais clientes satisfeitos do que insatisfeitos), utilizamos SMOTE para criar dados sintéticos da classe minoritária. Isso evita que o modelo aprenda apenas a responder 'Satisfeito' e ignore as falhas que causam a insatisfação.

In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter

print(f"Distribuição antes do SMOTE: {Counter(y_train)}")

smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_scaled, y_train)

print(f"Distribuição após o SMOTE: {Counter(y_train_res)}")

Distribuição antes do SMOTE: Counter({1: 67566, 0: 20443})
Distribuição após o SMOTE: Counter({1: 67566, 0: 67566})


## 2. Modelos

Fazendo uso do GridSearchCV para validação cruzada, 5 modelos foram treinados:

1. KNN: Baseado em proximidade; assume que experiências de compra similares geram notas similares.
2. Regressão Logística: Um classificador linear clássico que serve como base para verificar se o problema pode ser resolvido com uma separação simples entre as classes.
3. Árvore de Decisão: Modelo interpretável que mapeia decisões através de regras lógicas (se/então), facilitando a compreensão dos motivos da insatisfação.
4. Random Forest: Uma evolução da árvore de decisão que combina várias delas para reduzir erros e lidar melhor com a complexidade dos dados.
5. MLP (Rede Neural): Modelo capaz de capturar padrões matemáticos muito complexos e não-lineares que os outros algoritmos podem ignorar.

### 2.1. Modelo 1: KNN (K-Nearest Neighbors)

Hiperparâmetro: ajuste do n_neighbors para evitar overfitting ou generalização excessiva.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

knn = KNeighborsClassifier()

param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

grid_knn = GridSearchCV(knn, param_grid_knn, cv=5, scoring='f1', n_jobs=-1)

grid_knn.fit(X_train_res, y_train_res)

print(f"Melhores parâmetros para o KNN: {grid_knn.best_params_}")

Melhores parâmetros para o KNN: {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}


### 2.2. Modelo 2: Regressão Logística

Hiperparâmetro: ajuste de parâmetro C para que o modelo não decore demais os dados.

In [12]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver='liblinear', random_state=42)

# C menor = regularização mais forte
param_grid_lr = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l1', 'l2']
}

grid_lr = GridSearchCV(lr, param_grid_lr, cv=5, scoring='f1', n_jobs=-1)

grid_lr.fit(X_train_res, y_train_res)

print(f"Melhores parâmetros LR: {grid_lr.best_params_}")

Melhores parâmetros LR: {'C': 0.1, 'penalty': 'l1'}


### 2.3. Modelo 3: Árvore de Decisão

Hiperparâmetro: controle da max_depth para evitar que a árvore cresça indefinidamente.

In [13]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)

param_grid_dt = {
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'criterion': ['gini', 'entropy']
}

grid_dt = GridSearchCV(dt, param_grid_dt, cv=5, scoring='f1', n_jobs=-1)

grid_dt.fit(X_train_res, y_train_res)

print(f"Melhores parâmetros Árvore: {grid_dt.best_params_}")

Melhores parâmetros Árvore: {'criterion': 'entropy', 'max_depth': None, 'min_samples_split': 2}


### 2.4. Modelo 4: Random Forest

Hiperparâmetro: teste do n_estimators para equilibrar performance e custo computacional.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)

param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'criterion': ['gini', 'entropy']
}

grid_rf = GridSearchCV(rf, param_grid_rf, cv=5, scoring='f1', n_jobs=-1)

grid_rf.fit(X_train_res, y_train_res)

print(f"Melhores parâmetros Random Forest: {grid_rf.best_params_}")

Melhores parâmetros Random Forest: {'criterion': 'entropy', 'max_depth': None, 'n_estimators': 200}


### 2.5. Modelo 5: MLP

Hiperparâmetro: ajuste da hidden_layer_sizes para definir a capacidade de processamento da rede.

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(max_iter=500, random_state=42)

param_grid_mlp = {
    'hidden_layer_sizes': [(50,), (100,), (50, 20)],
    'activation': ['tanh', 'relu'],
    'learning_rate_init': [0.001, 0.01]
}

grid_mlp = GridSearchCV(mlp, param_grid_mlp, cv=5, scoring='f1', n_jobs=-1)

grid_mlp.fit(X_train_res, y_train_res)

print(f"Melhores parâmetros MLP: {grid_mlp.best_params_}")

Melhores parâmetros MLP: {'activation': 'relu', 'hidden_layer_sizes': (50, 20), 'learning_rate_init': 0.01}


## 3. Salvando Scaler e Modelos

Esta etapa garante a reprodutibilidade e modularidade do projeto, permitindo que os resultados do treinamento sejam "congelados" e reutilizados sem necessidade de reprocessamento.

Artefatos exportados (via joblib):

- Scaler para normalizar novos dados de entrada.
- Dicionário com os 5 modelos otimizados prontos para predição.
- Lista de atributos para manter a integridade e gerar gráficos de importância.
- Base de teste salvas em arquivos csv para consistência na avaliação entre notebooks.

In [18]:
import joblib
import os

path_models = '../../../models'

joblib.dump(scaler, os.path.join(path_models, 'scaler_olist.pkl'))

modelos = {
    "KNN": grid_knn,
    "Regressão Logística": grid_lr,
    "Árvore de Decisão": grid_dt,
    "Random Forest": grid_rf,
    "MLP (Rede Neural)": grid_mlp,
}

joblib.dump(modelos, os.path.join(path_models, 'modelos_treinados_olist.pkl'))

path_processed = '../../../data/processed/'

X_test.to_csv(os.path.join(path_processed, 'X_test.csv'), index=False)
y_test.to_csv(os.path.join(path_processed, 'y_test.csv'), index=False)
joblib.dump(features_list, os.path.join(path_models, 'features_list.pkl'))

['../../../models\\features_list.pkl']